In [1]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("D:\python_progs\Final_year_proj\Datasets\ppi_initial.csv")

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\abirb\AppData\Local\Temp\ipykernel_4660\496303123.py:1: SyntaxWarning: invalid escape sequence '\p'
  data=pd.read_csv("D:\python_progs\Final_year_proj\Datasets\ppi_initial.csv")


In [6]:
# Filter out rows that have no interaction data
data_interactions = data.dropna(subset=['Interacts with']).copy()

In [8]:
entry=data_interactions['Entry']
len(entry)

12823

In [9]:
#Defining the Edge Explosion Function
def explode_interactions(row):
    protein_a = row['Entry']
    interactors = row['Interacts with'].split('; ')
    
    edges = []
    for interactor in interactors:
        clean_interactor = interactor.strip()        
        if clean_interactor:
            edges.append((protein_a, clean_interactor))            
    return edges

In [10]:
# Apply the function to all rows to generate the list of all edges
all_edges = data_interactions.apply(explode_interactions, axis=1).sum()

In [11]:
#Creating the Edge List DataFrame
ppi_df = pd.DataFrame(all_edges, columns=['Protein_A_Entry', 'Protein_B_Entry'])

#Removing Self-Loops
ppi_df = ppi_df[ppi_df['Protein_A_Entry'] != ppi_df['Protein_B_Entry']]

#Remove Duplicate/Reciprocal Edges
ppi_df['Interaction'] = np.minimum(ppi_df['Protein_A_Entry'], ppi_df['Protein_B_Entry']) + \
                       '_' + np.maximum(ppi_df['Protein_A_Entry'], ppi_df['Protein_B_Entry'])

#Droping duplicates based on the unique ID, keeping only one entry per unique interaction
ppi_df = ppi_df.drop_duplicates(subset=['Interaction']).drop(columns=['Interaction'])

In [12]:
#Saving the Final Network Edge List
output_file = 'ppi_network_edges.csv'
ppi_df.to_csv(output_file, index=False)

In [ ]:
!pip install networkx

import pandas as pd
import networkx as nx
import os

def read_edge_list(path, src_col="Protein_A_Entry", dst_col="Protein_B_Entry"):
    df = pd.read_csv(path)
    if src_col not in df.columns or dst_col not in df.columns:
        raise ValueError(f"Input CSV must contain '{src_col}' and '{dst_col}' columns.")
    
    df = df[[src_col, dst_col]].dropna()
    df = df[df[src_col] != df[dst_col]]  # remove self-loops
    print(f"Loaded {len(df)} interactions from {path}")
    return df

#Building the PPI graph
def build_graph(edges_df, undirected=True):
    if undirected:
        G = nx.from_pandas_edgelist(edges_df, source=edges_df.columns[0], target=edges_df.columns[1])
    else:
        G = nx.from_pandas_edgelist(edges_df, source=edges_df.columns[0], target=edges_df.columns[1], create_using=nx.DiGraph)
    
    print(f"Graph built: {G.number_of_nodes()} proteins, {G.number_of_edges()} interactions")
    return G

#Computing centrality measures
def compute_centralities(G):
    print("Computing centrality measures...")

    degree = dict(G.degree())
    degree_centrality = nx.degree_centrality(G)
    betweenness = nx.betweenness_centrality(G)
    closeness = nx.closeness_centrality(G)
    eigenvector = nx.eigenvector_centrality(G, max_iter=1000)
    pagerank = nx.pagerank(G)
    clustering = nx.clustering(G)

    df = pd.DataFrame({
        "Protein": list(G.nodes()),
        "Degree": pd.Series(degree),
        "Degree_Centrality": pd.Series(degree_centrality),
        "Betweenness_Centrality": pd.Series(betweenness),
        "Closeness_Centrality": pd.Series(closeness),
        "Eigenvector_Centrality": pd.Series(eigenvector)
    })

    print("Centrality measures computed successfully.")
    return df

#Saving results to CSV
def save_results(df, output_path="./ppi_centrality_results.csv"):
    df.to_csv(output_path, index=False)
    print(f"Results saved to: {output_path}")

if __name__ == "__main__":
    
    input_path = r"D:\New folder\Apps\Jupyter Notebook\Project\ppi_network_edges.csv"  
    output_path = r"D:\New folder\Apps\Jupyter Notebook\Project\ppi_centrality_results.csv"

    edges_df = read_edge_list(input_path)
    G = build_graph(edges_df)
    centrality_df = compute_centralities(G)
    save_results(centrality_df, output_path)

  